# Remote actions and variables


Changing the state of remote resources


- Website: https://discovery.gitlabpages.inria.fr/enoslib/index.html
- Instant chat: https://framateam.org/enoslib


Prerequisites:
- Grid'5000 account
- A working EnOSlib environment and Jupyter (not included in EnOSlib dependencies, but `pip install jupyterlab` will install it)


In [1]:
import enoslib as en

# get some logging info
import logging
logging.basicConfig(level=logging.INFO)

## Setup on Grid'5000

### Describing the resources

In [5]:
# claim the resources
network = en.G5kNetworkConf(id="n1", type="prod", roles=["my_network"], site="rennes")

conf = (
    en.G5kConf.from_settings(job_type="allow_classic_ssh", job_name="rsd-01")
    .add_network_conf(network)
    .add_machine(
        roles=["control"], cluster="parasilo", nodes=1, primary_network=network
    )
    .add_machine(
        roles=["compute"],
        cluster="parasilo",
        nodes=1,
        primary_network=network,
    )
    .finalize()
)
conf

Conf@0x7f6e70218610
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "rsd-01",
    "job_type": "allow_classic_ssh",
    "key": "/home/msimonin/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "control"
                ],
                "primary_network": "n1",
                "secondary_networks": [],
                "cluster": "parasilo",
                "nodes": 1
            },
            {
                "roles": [
                    "compute"
                ],
                "primary_network": "n1",
                "secondary_networks": [],
                "cluster": "parasilo",
                "nodes": 1
            }
        ],
        "networks": [
            {
                "id": "n1",
                "type": "prod",
                "roles": [
                    "my_network"
                ],
                "site": "rennes"
            }
        ]
    }
}

### Reserving the resources

In [6]:
provider = en.G5k(conf)
roles, networks = provider.init()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Submitting {'name': 'rsd-01', 'types': ['allow_classic_ssh'], 'resources': "{cluster='parasilo'}/nodes=1+{cluster='parasilo'}/nodes=1,walltime=02:00:00", 'command': 'sleep 31536000', 'queue': 'default'} on rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1815745 on rennes [2021-08-24 17:13:18]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1815745 on rennes [2021-08-24 17:13:18]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:All jobs are Runni

Inspecting the ressources we own for the experiment's lifetime:

- roles: this is somehow a dictionnary whose keys are the role names and the associated values are the corresponding list of hosts
- networks: similar to roles but for networks

In [7]:
roles

{'control': [Host(address='parasilo-17.rennes.grid5000.fr', alias='parasilo-17.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())], 'compute': [Host(address='parasilo-24.rennes.grid5000.fr', alias='parasilo-24.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]}

In [8]:
# list of host on a given role
roles["control"]

[Host(address='parasilo-17.rennes.grid5000.fr', alias='parasilo-17.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]

In [9]:
# a single host
roles["control"][0]

Host(address='parasilo-17.rennes.grid5000.fr', alias='parasilo-17.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())

In [10]:
networks

{'my_network': [<enoslib.infra.enos_g5k.objects.G5kEnosProd4Network object at 0x7f6e70234650>, <enoslib.infra.enos_g5k.objects.G5kEnosProd6Network object at 0x7f6e6ff27a90>]}

In [11]:
# sync some more information in the host data structure (for illustration purpose here)
en.sync_info(roles, networks)

INFO:enoslib.api:extra_vars = {'ansible_python_interpreter': 'python3'}
[WARNING]: No inventory was parsed, only implicit localhost is available



PLAY [all] ************************************************************************************************************************************************

TASK [hostname] *******************************************************************************************************************************************
 [started TASK: hostname on parasilo-17.rennes.grid5000.fr]
 [started TASK: hostname on parasilo-24.rennes.grid5000.fr]
changed: [parasilo-24.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]


INFO:enoslib.api:extra_vars = {'enos_action': 'check_network', 'facts_file': '/home/msimonin/workspace/repos/enoslib/_tmp_enos_/facts.json', 'ansible_python_interpreter': 'python3'}



PLAY [Gather facts for all hosts] *************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [parasilo-17.rennes.grid5000.fr]
ok: [parasilo-24.rennes.grid5000.fr]

TASK [setup] **********************************************************************************************************************************************
ok: [parasilo-17.rennes.grid5000.fr]
ok: [parasilo-24.rennes.grid5000.fr]

PLAY [Utils functions] ************************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [parasilo-17.rennes.grid5000.fr]
ok: [parasilo-24.rennes.grid50

ip
::1/128
127.0.0.1/8
ip
fe80::eef4:bbff:fed1:1b8/64
172.16.97.17/20
ip
::1/128
127.0.0.1/8
ip
fe80::eef4:bbff:fed1:1018/64


## Acting on remote nodes

### run a command, filter results

In [12]:
results = en.run_command("cat /etc/apt/sources.list", roles=roles)
results

INFO:enoslib.api:extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [cat /etc/apt/sources.list] **************************************************************************************************************************
 [started TASK: cat /etc/apt/sources.list on parasilo-17.rennes.grid5000.fr]
 [started TASK: cat /etc/apt/sources.list on parasilo-24.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]
changed: [parasilo-24.rennes.grid5000.fr]


CommandResult(host='parasilo-17.rennes.grid5000.fr', task='cat /etc/apt/sources.list', status='OK', payload={'cmd': 'cat /etc/apt/sources.list', 'stdout': 'deb http://deb.debian.org/debian buster main contrib non-free\ndeb-src http://deb.debian.org/debian buster main contrib non-free\ndeb http://deb.debian.org/debian  buster-updates main contrib non-free\ndeb-src http://deb.debian.org/debian buster-updates main contrib non-free\ndeb http://security.debian.org/ buster/updates main contrib non-free\ndeb-src http://security.debian.org/ buster/updates main contrib non-free\ndeb http://deb.debian.org/debian buster-backports main contrib non-free\ndeb-src http://deb.debian.org/debian buster-backports main contrib non-free', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:13:58.997119', 'end': '2021-08-24 17:13:58.999810', 'delta': '0:00:00.002691', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'cat /etc/apt/sources.list', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['deb http://deb.debian.org/debian buster main contrib non-free', 'deb-src http://deb.debian.org/debian buster main contrib non-free', 'deb http://deb.debian.org/debian  buster-updates main contrib non-free', 'deb-src http://deb.debian.org/debian buster-updates main contrib non-free', 'deb http://security.debian.org/ buster/updates main contrib non-free', 'deb-src http://security.debian.org/ buster/updates main contrib non-free', 'deb http://deb.debian.org/debian buster-backports main contrib non-free', 'deb-src http://deb.debian.org/debian buster-backports main contrib non-free'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-24.rennes.grid5000.fr', task='cat /etc/apt/sources.list', status='OK', payload={'cmd': 'cat /etc/apt/sources.list', 'stdout': 'deb http://deb.debian.org/debian buster main contrib non-free\ndeb-src http://deb.debian.org/debian buster main contrib non-free\ndeb http://deb.debian.org/debian  buster-updates main contrib non-free\ndeb-src http://deb.debian.org/debian buster-updates main contrib non-free\ndeb http://security.debian.org/ buster/updates main contrib non-free\ndeb-src http://security.debian.org/ buster/updates main contrib non-free\ndeb http://deb.debian.org/debian buster-backports main contrib non-free\ndeb-src http://deb.debian.org/debian buster-backports main contrib non-free', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:13:59.006244', 'end': '2021-08-24 17:13:59.008895', 'delta': '0:00:00.002651', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'cat /etc/apt/sources.list', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['deb http://deb.debian.org/debian buster main contrib non-free', 'deb-src http://deb.debian.org/debian buster main contrib non-free', 'deb http://deb.debian.org/debian  buster-updates main contrib non-free', 'deb-src http://deb.debian.org/debian buster-updates main contrib non-free', 'deb http://security.debian.org/ buster/updates main contrib non-free', 'deb-src http://security.debian.org/ buster/updates main contrib non-free', 'deb http://deb.debian.org/debian buster-backports main contrib non-free', 'deb-src http://deb.debian.org/debian buster-backports main contrib non-free'], 'stderr_lines': [], '_ansible_no_log': False})

In [13]:
one_result = results.filter(host=roles["control"][0].alias)[0]
one_result

CommandResult(host='parasilo-17.rennes.grid5000.fr', task='cat /etc/apt/sources.list', status='OK', payload={'cmd': 'cat /etc/apt/sources.list', 'stdout': 'deb http://deb.debian.org/debian buster main contrib non-free\ndeb-src http://deb.debian.org/debian buster main contrib non-free\ndeb http://deb.debian.org/debian  buster-updates main contrib non-free\ndeb-src http://deb.debian.org/debian buster-updates main contrib non-free\ndeb http://security.debian.org/ buster/updates main contrib non-free\ndeb-src http://security.debian.org/ buster/updates main contrib non-free\ndeb http://deb.debian.org/debian buster-backports main contrib non-free\ndeb-src http://deb.debian.org/debian buster-backports main contrib non-free', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:13:58.997119', 'end': '2021-08-24 17:13:58.999810', 'delta': '0:00:00.002691', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'cat /etc/apt/sources.list', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['deb http://deb.debian.org/debian buster main contrib non-free', 'deb-src http://deb.debian.org/debian buster main contrib non-free', 'deb http://deb.debian.org/debian  buster-updates main contrib non-free', 'deb-src http://deb.debian.org/debian buster-updates main contrib non-free', 'deb http://security.debian.org/ buster/updates main contrib non-free', 'deb-src http://security.debian.org/ buster/updates main contrib non-free', 'deb http://deb.debian.org/debian buster-backports main contrib non-free', 'deb-src http://deb.debian.org/debian buster-backports main contrib non-free'], 'stderr_lines': [], '_ansible_no_log': False})

In [14]:
one_result.payload["stdout"]

'deb http://deb.debian.org/debian buster main contrib non-free\ndeb-src http://deb.debian.org/debian buster main contrib non-free\ndeb http://deb.debian.org/debian  buster-updates main contrib non-free\ndeb-src http://deb.debian.org/debian buster-updates main contrib non-free\ndeb http://security.debian.org/ buster/updates main contrib non-free\ndeb-src http://security.debian.org/ buster/updates main contrib non-free\ndeb http://deb.debian.org/debian buster-backports main contrib non-free\ndeb-src http://deb.debian.org/debian buster-backports main contrib non-free'

There are some specific shortcuts when the remote actions is a remote (shell) command: `.stdout`, `.stderr`, `.rc`

In [15]:
print(f"stdout = {one_result.stdout}\n", f"stderr={one_result.stderr}\n", f"return code = {one_result.rc}")

stdout = deb http://deb.debian.org/debian buster main contrib non-free
deb-src http://deb.debian.org/debian buster main contrib non-free
deb http://deb.debian.org/debian  buster-updates main contrib non-free
deb-src http://deb.debian.org/debian buster-updates main contrib non-free
deb http://security.debian.org/ buster/updates main contrib non-free
deb-src http://security.debian.org/ buster/updates main contrib non-free
deb http://deb.debian.org/debian buster-backports main contrib non-free
deb-src http://deb.debian.org/debian buster-backports main contrib non-free
 stderr=
 return code = 0


### Filtering hosts on which the command is run

`run_command` acts on remote hosts. Those hosts can be given as a `Roles` type (output of `provider.init`) or as a list of `Host` or a single `Host`. 


In [16]:
# some roles
en.run_command("date", roles = roles)

INFO:enoslib.api:extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [date] ***********************************************************************************************************************************************
 [started TASK: date on parasilo-17.rennes.grid5000.fr]
 [started TASK: date on parasilo-24.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]
changed: [parasilo-24.rennes.grid5000.fr]


CommandResult(host='parasilo-17.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Tue 24 Aug 2021 05:14:11 PM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:14:11.041558', 'end': '2021-08-24 17:14:11.044053', 'delta': '0:00:00.002495', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Tue 24 Aug 2021 05:14:11 PM CEST'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-24.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Tue 24 Aug 2021 05:14:11 PM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:14:11.050523', 'end': '2021-08-24 17:14:11.052991', 'delta': '0:00:00.002468', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Tue 24 Aug 2021 05:14:11 PM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

In [17]:
# a list of hosts
en.run_command("date", roles = roles["control"])

INFO:enoslib.api:extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [date] ***********************************************************************************************************************************************
 [started TASK: date on parasilo-17.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]


CommandResult(host='parasilo-17.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Tue 24 Aug 2021 05:14:24 PM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:14:24.781335', 'end': '2021-08-24 17:14:24.783784', 'delta': '0:00:00.002449', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Tue 24 Aug 2021 05:14:24 PM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

In [18]:
# a single host
en.run_command("date", roles=roles["control"][0])

INFO:enoslib.api:extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [date] ***********************************************************************************************************************************************
 [started TASK: date on parasilo-17.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]


CommandResult(host='parasilo-17.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Tue 24 Aug 2021 05:14:29 PM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:14:29.312216', 'end': '2021-08-24 17:14:29.314718', 'delta': '0:00:00.002502', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Tue 24 Aug 2021 05:14:29 PM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

### Remote actions

Tools like Ansible, Puppet, Chef, Terraform ... are shipped with a set of predefined remote actions to ease the administrator life.

Actions like copying file, adding some users, managing packages, making sure a line is absent from a configuration file, managing docker containers ... are first-class citizens actions and brings some nice garantees like correctness and idempotency.

There are 1000+ modules  available:
https://docs.ansible.com/ansible/2.9/modules/list_of_all_modules.html

---

EnOSlib wraps Ansible module and let you use them from Python (without writting any YAML file). You can call any module by using the `actions` context manager:

In the following we install docker (using g5k provided script) and a docker container. We also need to install the python docker binding on the remote machine so that Ansible can interact with the docker daemons on the remote machines. This block of actions is idempotent.


In [19]:
with en.actions(roles=roles) as a:
    # prepending with `which docker ||` makes this task idempotent
    a.shell("which docker || /grid5000/code/bin/g5k-setup-docker")
    a.pip(name="docker", state="present")
    a.docker_container(image="nginx", state="started", name="my_webserver")
    # we can capture the outputs if we really want to
    results = a.results
results

INFO:enoslib.api:extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [which docker || /grid5000/code/bin/g5k-setup-docker] ************************************************************************************************
 [started TASK: which docker || /grid5000/code/bin/g5k-setup-docker on parasilo-17.rennes.grid5000.fr]
 [started TASK: which docker || /grid5000/code/bin/g5k-setup-docker on parasilo-24.rennes.grid5000.fr]
changed: [parasilo-24.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]

TASK [pip] ************************************************************************************************************************************************
 [started TASK: pip on parasilo-17.rennes.grid5000.fr]
 [started TASK: pip on parasilo-24.rennes.grid5000.fr]
changed: [parasilo-24.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]

TASK [docker_container] ********************

[DEPRECATION WARNING]: The container_default_behavior option will change its default value from "compatibility" to "no_defaults" in community.docker 
2.0.0. To remove this warning, please specify an explicit value for it now. This feature will be removed from community.docker in version 2.0.0. 
Deprecation warnings can be disabled by setting deprecation_warnings=False in ansible.cfg.


changed: [parasilo-17.rennes.grid5000.fr]
changed: [parasilo-24.rennes.grid5000.fr]


CommandResult(host='parasilo-24.rennes.grid5000.fr', task='which docker || /grid5000/code/bin/g5k-setup-docker', status='OK', payload={'cmd': 'which docker || /grid5000/code/bin/g5k-setup-docker', 'stdout': 'curl -sSL https://get.docker.com/ | sh\nsudo mkdir -p /etc/docker\necho "{ \\"registry-mirrors\\": [\\"http://docker-cache.grid5000.fr\\"] }" | sudo tee /etc/docker/daemon.json\nsudo systemctl restart docker\nsudo chmod o+rw /var/run/docker.sock', 'stderr': '+ sh -c apt-get update -qq >/dev/null\nW: An error occurred during the signature verification. The repository is not updated and the previous index files will be used. GPG error: https://repo.radeon.com/rocm/apt/debian xenial InRelease: The following signatures were invalid: EXPKEYSIG 9386B48A1A693C5C James Adrian Edwards (ROCm Release Manager) <JamesAdrian.Edwards@amd.com>\nW: Failed to fetch https://repo.radeon.com/rocm/apt/debian/dists/xenial/InRelease  The following signatures were invalid: EXPKEYSIG 9386B48A1A693C5C James Adrian Edwards (ROCm Release Manager) <JamesAdrian.Edwards@amd.com>\nW: Some index files failed to download. They have been ignored, or old ones used instead.\n+ sh -c DEBIAN_FRONTEND=noninteractive apt-get install -y -qq apt-transport-https ca-certificates curl >/dev/null\n+ sh -c curl -fsSL "https://download.docker.com/linux/debian/gpg" | gpg --dearmor --yes -o /usr/share/keyrings/docker-archive-keyring.gpg\n+ sh -c echo "deb [arch=amd64 signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/debian buster stable" > /etc/apt/sources.list.d/docker.list\n+ sh -c apt-get update -qq >/dev/null\nW: An error occurred during the signature verification. The repository is not updated and the previous index files will be used. GPG error: https://repo.radeon.com/rocm/apt/debian xenial InRelease: The following signatures were invalid: EXPKEYSIG 9386B48A1A693C5C James Adrian Edwards (ROCm Release Manager) <JamesAdrian.Edwards@amd.com>\nW: Failed to fetch https://repo.radeon.com/rocm/apt/debian/dists/xenial/InRelease  The following signatures were invalid: EXPKEYSIG 9386B48A1A693C5C James Adrian Edwards (ROCm Release Manager) <JamesAdrian.Edwards@amd.com>\nW: Some index files failed to download. They have been ignored, or old ones used instead.\n+ sh -c DEBIAN_FRONTEND=noninteractive apt-get install -y -qq --no-install-recommends  docker-ce-cli docker-scan-plugin docker-ce >/dev/null\n+ version_gte 20.10\n+ [ -z  ]\n+ return 0\n+ sh -c DEBIAN_FRONTEND=noninteractive apt-get install -y -qq docker-ce-rootless-extras >/dev/null\n+ sh -c docker version', 'rc': 0, 'start': '2021-08-24 17:14:35.800313', 'end': '2021-08-24 17:15:05.871384', 'delta': '0:00:30.071071', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'which docker || /grid5000/code/bin/g5k-setup-docker', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['curl -sSL https://get.docker.com/ | sh', 'sudo mkdir -p /etc/docker', 'echo "{ \\"registry-mirrors\\": [\\"http://docker-cache.grid5000.fr\\"] }" | sudo tee /etc/docker/daemon.json', 'sudo systemctl restart docker', 'sudo chmod o+rw /var/run/docker.sock'], 'stderr_lines': ['+ sh -c apt-get update -qq >/dev/null', 'W: An error occurred during the signature verification. The repository is not updated and the previous index files will be used. GPG error: https://repo.radeon.com/rocm/apt/debian xenial InRelease: The following signatures were invalid: EXPKEYSIG 9386B48A1A693C5C James Adrian Edwards (ROCm Release Manager) <JamesAdrian.Edwards@amd.com>', 'W: Failed to fetch https://repo.radeon.com/rocm/apt/debian/dists/xenial/InRelease  The following signatures were invalid: EXPKEYSIG 9386B48A1A693C5C James Adrian Edwards (ROCm Release Manager) <JamesAdrian.Edwards@amd.com>', 'W: Some index files failed to download. They have been ignored, or old ones used

In [20]:
results.filter(task="docker_container")[0]

CustomCommandResult(host='parasilo-17.rennes.grid5000.fr', task='docker_container', status='OK', payload={'changed': True, 'container': {'Id': 'b2bf90f48f4f20a75b124c6f03c724a999d07c9db6a19443f3fa2cf85516fdce', 'Created': '2021-08-24T15:15:16.653171818Z', 'Path': '/docker-entrypoint.sh', 'Args': ['nginx', '-g', 'daemon off;'], 'State': {'Status': 'running', 'Running': True, 'Paused': False, 'Restarting': False, 'OOMKilled': False, 'Dead': False, 'Pid': 14720, 'ExitCode': 0, 'Error': '', 'StartedAt': '2021-08-24T15:15:19.434872036Z', 'FinishedAt': '0001-01-01T00:00:00Z'}, 'Image': 'sha256:dd34e67e3371dc2d1328790c3157ee42dfcae74afffd86b297459ed87a98c0fb', 'ResolvConfPath': '/var/lib/docker/containers/b2bf90f48f4f20a75b124c6f03c724a999d07c9db6a19443f3fa2cf85516fdce/resolv.conf', 'HostnamePath': '/var/lib/docker/containers/b2bf90f48f4f20a75b124c6f03c724a999d07c9db6a19443f3fa2cf85516fdce/hostname', 'HostsPath': '/var/lib/docker/containers/b2bf90f48f4f20a75b124c6f03c724a999d07c9db6a19443f3fa2cf85516fdce/hosts', 'LogPath': '/var/lib/docker/containers/b2bf90f48f4f20a75b124c6f03c724a999d07c9db6a19443f3fa2cf85516fdce/b2bf90f48f4f20a75b124c6f03c724a999d07c9db6a19443f3fa2cf85516fdce-json.log', 'Name': '/my_webserver', 'RestartCount': 0, 'Driver': 'overlay2', 'Platform': 'linux', 'MountLabel': '', 'ProcessLabel': '', 'AppArmorProfile': 'docker-default', 'ExecIDs': None, 'HostConfig': {'Binds': [], 'ContainerIDFile': '', 'LogConfig': {'Type': 'json-file', 'Config': {}}, 'NetworkMode': 'default', 'PortBindings': None, 'RestartPolicy': {'Name': '', 'MaximumRetryCount': 0}, 'AutoRemove': False, 'VolumeDriver': '', 'VolumesFrom': None, 'CapAdd': None, 'CapDrop': None, 'CgroupnsMode': 'host', 'Dns': None, 'DnsOptions': None, 'DnsSearch': None, 'ExtraHosts': None, 'GroupAdd': None, 'IpcMode': 'private', 'Cgroup': '', 'Links': None, 'OomScoreAdj': 0, 'PidMode': '', 'Privileged': False, 'PublishAllPorts': False, 'ReadonlyRootfs': False, 'SecurityOpt': None, 'UTSMode': '', 'UsernsMode': '', 'ShmSize': 67108864, 'Runtime': 'runc', 'ConsoleSize': [0, 0], 'Isolation': '', 'CpuShares': 0, 'Memory': 0, 'NanoCpus': 0, 'CgroupParent': '', 'BlkioWeight': 0, 'BlkioWeightDevice': None, 'BlkioDeviceReadBps': None, 'BlkioDeviceWriteBps': None, 'BlkioDeviceReadIOps': None, 'BlkioDeviceWriteIOps': None, 'CpuPeriod': 0, 'CpuQuota': 0, 'CpuRealtimePeriod': 0, 'CpuRealtimeRuntime': 0, 'CpusetCpus': '', 'CpusetMems': '', 'Devices': None, 'DeviceCgroupRules': None, 'DeviceRequests': None, 'KernelMemory': 0, 'KernelMemoryTCP': 0, 'MemoryReservation': 0, 'MemorySwap': 0, 'MemorySwappiness': None, 'OomKillDisable': False, 'PidsLimit': None, 'Ulimits': None, 'CpuCount': 0, 'CpuPercent': 0, 'IOMaximumIOps': 0, 'IOMaximumBandwidth': 0, 'MaskedPaths': ['/proc/asound', '/proc/acpi', '/proc/kcore', '/proc/keys', '/proc/latency_stats', '/proc/timer_list', '/proc/timer_stats', '/proc/sched_debug', '/proc/scsi', '/sys/firmware'], 'ReadonlyPaths': ['/proc/bus', '/proc/fs', '/proc/irq', '/proc/sys', '/proc/sysrq-trigger'], 'Init': False}, 'GraphDriver': {'Data': {'LowerDir': '/var/lib/docker/overlay2/c2262636665c2019ef552f8a15e5d10bcd3269103a7cf56591bd66c9683f4d1b-init/diff:/var/lib/docker/overlay2/b309b755d9ccbd962d1fa569ac24305182e792c0d856b382baf1148ea8f0d8cd/diff:/var/lib/docker/overlay2/c10519ef34f5c0ff8d8358d9bcf5ef5a6311f03b2903532759787e485450428a/diff:/var/lib/docker/overlay2/e8d94c857640f7ce2ec2214516b00bf53cb05ddfca8e19047dd87109c78880af/diff:/var/lib/docker/overlay2/2fcc44c10530f9c0b2616600ccb40c654fc355c834e1d812d2d0492f0ec97ddc/diff:/var/lib/docker/overlay2/8d4d0df6e602c7ade1c64221c19bbe38082176c42e45a38cab14de32cdfe141d/diff:/var/lib/docker/overlay2/1ada2f61beac7647403e96de6e0842b5e07ec8c8b8a9be8e017e126fe0a7d050/diff', 'MergedDir': '/var/lib/docker/overlay2/c2262636665c2019ef552f8a15e5d10bcd3269103a7cf56591bd66c9683f4d1b/merged', 'UpperDir': '/var/lib/docker/overlay2/c2262636665c2019ef552f8a15e5d10bcd3269103a7cf56591bd66c9683f4d1b/diff', 'WorkDir': '/var/lib/docker/

### Background actions

Sometime you need to fire a process on some remote machines that needs to survive the remote connection that started it. EnOSlib provides a `keyword` argument for this purpose and can be used when calling modules (when supported).

In [21]:
# synchronous execution, will wait until the end of the shell command
results = en.run_command("for i in $(seq 1 10); do sleep 1; echo toto; done", roles=roles)
results

INFO:enoslib.api:extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [for i in $(seq 1 10); do sleep 1; echo toto; done] **************************************************************************************************
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-17.rennes.grid5000.fr]
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-24.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]
changed: [parasilo-24.rennes.grid5000.fr]


CommandResult(host='parasilo-17.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'cmd': 'for i in $(seq 1 10); do sleep 1; echo toto; done', 'stdout': 'toto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:15:50.628843', 'end': '2021-08-24 17:16:00.648062', 'delta': '0:00:10.019219', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'for i in $(seq 1 10); do sleep 1; echo toto; done', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-24.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'cmd': 'for i in $(seq 1 10); do sleep 1; echo toto; done', 'stdout': 'toto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:15:50.654538', 'end': '2021-08-24 17:16:00.672614', 'delta': '0:00:10.018076', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'for i in $(seq 1 10); do sleep 1; echo toto; done', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto'], 'stderr_lines': [], '_ansible_no_log': False})

In [23]:
# The remote command will be daemonize on the remote machines
results = en.run_command("for i in $(seq 1 10); do sleep 1; echo toto; done", roles=roles, background=True)
results

INFO:enoslib.api:extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [for i in $(seq 1 10); do sleep 1; echo toto; done] **************************************************************************************************
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-17.rennes.grid5000.fr]
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-24.rennes.grid5000.fr]
changed: [parasilo-24.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]


AsyncCommandResult(host='parasilo-24.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'started': 1, 'finished': 0, 'ansible_job_id': '22525073620.27377', 'results_file': '/root/.ansible_async/22525073620.27377', 'changed': True, '_ansible_no_log': False})
AsyncCommandResult(host='parasilo-17.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'started': 1, 'finished': 0, 'ansible_job_id': '944605924460.14911', 'results_file': '/root/.ansible_async/944605924460.14911', 'changed': True, '_ansible_no_log': False})

In [24]:
# you can get back the status of the daemonized process by reading the remote results_file
h  = roles["control"][0]
result_file = results.filter(host=h.alias)[0].results_file
cat_result = en.run_command(f"cat {result_file}",roles=h)
cat_result

INFO:enoslib.api:extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [cat /root/.ansible_async/944605924460.14911] ********************************************************************************************************
 [started TASK: cat /root/.ansible_async/944605924460.14911 on parasilo-17.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]


CommandResult(host='parasilo-17.rennes.grid5000.fr', task='cat /root/.ansible_async/944605924460.14911', status='OK', payload={'cmd': 'cat /root/.ansible_async/944605924460.14911', 'stdout': '{"cmd": "for i in $(seq 1 10); do sleep 1; echo toto; done", "stdout": "toto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto", "stderr": "", "rc": 0, "start": "2021-08-24 17:16:16.207795", "end": "2021-08-24 17:16:26.218495", "delta": "0:00:10.010700", "changed": true, "invocation": {"module_args": {"_raw_params": "for i in $(seq 1 10); do sleep 1; echo toto; done", "_uses_shell": true, "warn": true, "stdin_add_newline": true, "strip_empty_ends": true, "argv": null, "chdir": null, "executable": null, "creates": null, "removes": null, "stdin": null}}}', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:16:46.747535', 'end': '2021-08-24 17:16:46.750084', 'delta': '0:00:00.002549', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'cat /root/.ansible_async/944605924460.14911', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['{"cmd": "for i in $(seq 1 10); do sleep 1; echo toto; done", "stdout": "toto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto", "stderr": "", "rc": 0, "start": "2021-08-24 17:16:16.207795", "end": "2021-08-24 17:16:26.218495", "delta": "0:00:10.010700", "changed": true, "invocation": {"module_args": {"_raw_params": "for i in $(seq 1 10); do sleep 1; echo toto; done", "_uses_shell": true, "warn": true, "stdin_add_newline": true, "strip_empty_ends": true, "argv": null, "chdir": null, "executable": null, "creates": null, "removes": null, "stdin": null}}}'], 'stderr_lines': [], '_ansible_no_log': False})

In [25]:
# the result_file content is json encoded so decoding it
import json
print(json.loads(cat_result[0].stdout)["stdout"])

toto
toto
toto
toto
toto
toto
toto
toto
toto
toto


## Using variables

### Same variable value for everyone

Nothing surprising here, you can use regular python interpolation (e.g a `f-string`).
String are interpolated by the interpreter before being manipulated.

In [31]:
host_to_ping = roles["control"][0].alias
host_to_ping

results = en.run_command(f"ping -c 5 {host_to_ping}", roles=roles)
results

INFO:enoslib.api:extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [ping -c 5 parasilo-17.rennes.grid5000.fr] ***********************************************************************************************************
 [started TASK: ping -c 5 parasilo-17.rennes.grid5000.fr on parasilo-17.rennes.grid5000.fr]
 [started TASK: ping -c 5 parasilo-17.rennes.grid5000.fr on parasilo-24.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]
changed: [parasilo-24.rennes.grid5000.fr]


CommandResult(host='parasilo-17.rennes.grid5000.fr', task='ping -c 5 parasilo-17.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-17.rennes.grid5000.fr', 'stdout': 'PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.035 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=2 ttl=64 time=0.014 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=3 ttl=64 time=0.031 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=4 ttl=64 time=0.032 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=5 ttl=64 time=0.031 ms\n\n--- parasilo-17.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 87ms\nrtt min/avg/max/mdev = 0.014/0.028/0.035/0.009 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:21:12.043346', 'end': '2021-08-24 17:21:16.131011', 'delta': '0:00:04.087665', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-17.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.035 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=2 ttl=64 time=0.014 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=3 ttl=64 time=0.031 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=4 ttl=64 time=0.032 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=5 ttl=64 time=0.031 ms', '', '--- parasilo-17.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 87ms', 'rtt min/avg/max/mdev = 0.014/0.028/0.035/0.009 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-24.rennes.grid5000.fr', task='ping -c 5 parasilo-17.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-17.rennes.grid5000.fr', 'stdout': 'PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.188 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=2 ttl=64 time=0.192 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=3 ttl=64 time=0.203 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=4 ttl=64 time=0.128 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=5 ttl=64 time=0.146 ms\n\n--- parasilo-17.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 95ms\nrtt min/avg/max/mdev = 0.128/0.171/0.203/0.031 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:21:12.076227', 'end': '2021-08-24 17:21:16.172052', 'delta': '0:00:04.095825', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-17.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.188 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=2 ttl=64 time=0.192 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=3 ttl=64 time=0.203 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=4 ttl=64 time=0.128 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=5 ttl=

In [39]:
[(r.host, r.stdout) for r in results]

[('parasilo-17.rennes.grid5000.fr',
  'PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.035 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=2 ttl=64 time=0.014 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=3 ttl=64 time=0.031 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=4 ttl=64 time=0.032 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=5 ttl=64 time=0.031 ms\n\n--- parasilo-17.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 87ms\nrtt min/avg/max/mdev = 0.014/0.028/0.035/0.009 ms'),
 ('parasilo-24.rennes.grid5000.fr',
  'PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.188 ms\n64 bytes from parasilo-17.rennes.grid

### Using templates / Ansible variables

There's an alternative way to pass a variable to a task: using `extra_vars`.
The difference with the previous case (python interpreted variables) is the fact that the variable is interpolated right before execution happens on the remote node.
One could imagine the the value is broadcasted to all nodes and replaced right before the execution.

To indicate that we want to use this kind of variables, we need to pass its value using the `extra_vars` dictionnary and use a template (`{{ ... }}`) in the task description.

In [41]:
host_to_ping = roles["control"][0].alias
host_to_ping

results = en.run_command("ping -c 5 {{ my_template_variable }}", roles=roles, extra_vars=dict(my_template_variable=host_to_ping))
results

INFO:enoslib.api:extra_vars = {'my_template_variable': 'parasilo-17.rennes.grid5000.fr', 'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [ping -c 5 parasilo-17.rennes.grid5000.fr] ***********************************************************************************************************
 [started TASK: ping -c 5 {{ my_template_variable }} on parasilo-17.rennes.grid5000.fr]
 [started TASK: ping -c 5 {{ my_template_variable }} on parasilo-24.rennes.grid5000.fr]
changed: [parasilo-24.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]


CommandResult(host='parasilo-24.rennes.grid5000.fr', task='ping -c 5 parasilo-17.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-17.rennes.grid5000.fr', 'stdout': 'PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.143 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=2 ttl=64 time=0.188 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=3 ttl=64 time=0.188 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=4 ttl=64 time=0.158 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=5 ttl=64 time=0.091 ms\n\n--- parasilo-17.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 89ms\nrtt min/avg/max/mdev = 0.091/0.153/0.188/0.038 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:30:34.258643', 'end': '2021-08-24 17:30:38.347948', 'delta': '0:00:04.089305', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-17.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.143 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=2 ttl=64 time=0.188 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=3 ttl=64 time=0.188 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=4 ttl=64 time=0.158 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=5 ttl=64 time=0.091 ms', '', '--- parasilo-17.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 89ms', 'rtt min/avg/max/mdev = 0.091/0.153/0.188/0.038 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-17.rennes.grid5000.fr', task='ping -c 5 parasilo-17.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-17.rennes.grid5000.fr', 'stdout': 'PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.022 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=2 ttl=64 time=0.032 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=3 ttl=64 time=0.031 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=4 ttl=64 time=0.032 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=5 ttl=64 time=0.014 ms\n\n--- parasilo-17.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 94ms\nrtt min/avg/max/mdev = 0.014/0.026/0.032/0.007 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:30:34.308295', 'end': '2021-08-24 17:30:38.402891', 'delta': '0:00:04.094596', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-17.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.022 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=2 ttl=64 time=0.032 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=3 ttl=64 time=0.031 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=4 ttl=64 time=0.032 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=5 ttl=

### Host specific variables

In the above, we've seen how a common value can be broadcasted to all remote nodes.  What if we want host specific value ?

For instance in our case we'd like `host 1` to ping `host 2` and `host 2` to ping `host 1`. That make the `host_to_ping` variable host-specific.

For this purpose you can use the `extra` attribute of the `Host` objects and use a template as before.

In [43]:
control_host = roles["control"][0]
compute_host = roles["compute"][0]
control_host.extra.update(host_to_ping=compute_host.address)
compute_host.extra.update(host_to_ping=control_host.address)

> Note that the `extra` attribute is mutable :(

In [47]:
results = en.run_command("ping -c 5 {{ host_to_ping }}", roles=roles)
results

INFO:enoslib.api:extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [ping -c 5 parasilo-24.rennes.grid5000.fr] ***********************************************************************************************************
 [started TASK: ping -c 5 {{ host_to_ping }} on parasilo-17.rennes.grid5000.fr]
 [started TASK: ping -c 5 {{ host_to_ping }} on parasilo-24.rennes.grid5000.fr]
changed: [parasilo-17.rennes.grid5000.fr]
changed: [parasilo-24.rennes.grid5000.fr]


CommandResult(host='parasilo-17.rennes.grid5000.fr', task='ping -c 5 parasilo-24.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-24.rennes.grid5000.fr', 'stdout': 'PING parasilo-24.rennes.grid5000.fr (172.16.97.24) 56(84) bytes of data.\n64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=1 ttl=64 time=0.172 ms\n64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=2 ttl=64 time=0.113 ms\n64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=3 ttl=64 time=0.159 ms\n64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=4 ttl=64 time=0.193 ms\n64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=5 ttl=64 time=0.166 ms\n\n--- parasilo-24.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 91ms\nrtt min/avg/max/mdev = 0.113/0.160/0.193/0.029 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:38:44.999426', 'end': '2021-08-24 17:38:49.091165', 'delta': '0:00:04.091739', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-24.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-24.rennes.grid5000.fr (172.16.97.24) 56(84) bytes of data.', '64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=1 ttl=64 time=0.172 ms', '64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=2 ttl=64 time=0.113 ms', '64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=3 ttl=64 time=0.159 ms', '64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=4 ttl=64 time=0.193 ms', '64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=5 ttl=64 time=0.166 ms', '', '--- parasilo-24.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 91ms', 'rtt min/avg/max/mdev = 0.113/0.160/0.193/0.029 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-24.rennes.grid5000.fr', task='ping -c 5 parasilo-17.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-17.rennes.grid5000.fr', 'stdout': 'PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.105 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=2 ttl=64 time=0.154 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=3 ttl=64 time=0.139 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=4 ttl=64 time=0.189 ms\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=5 ttl=64 time=0.092 ms\n\n--- parasilo-17.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 99ms\nrtt min/avg/max/mdev = 0.092/0.135/0.189/0.037 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-24 17:38:45.000970', 'end': '2021-08-24 17:38:49.100012', 'delta': '0:00:04.099042', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-17.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.105 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=2 ttl=64 time=0.154 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=3 ttl=64 time=0.139 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=4 ttl=64 time=0.189 ms', '64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=5 ttl=

In [48]:
[(r.host, r.stdout) for r in results]

[('parasilo-17.rennes.grid5000.fr',
  'PING parasilo-24.rennes.grid5000.fr (172.16.97.24) 56(84) bytes of data.\n64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=1 ttl=64 time=0.172 ms\n64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=2 ttl=64 time=0.113 ms\n64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=3 ttl=64 time=0.159 ms\n64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=4 ttl=64 time=0.193 ms\n64 bytes from parasilo-24.rennes.grid5000.fr (172.16.97.24): icmp_seq=5 ttl=64 time=0.166 ms\n\n--- parasilo-24.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 91ms\nrtt min/avg/max/mdev = 0.113/0.160/0.193/0.029 ms'),
 ('parasilo-24.rennes.grid5000.fr',
  'PING parasilo-17.rennes.grid5000.fr (172.16.97.17) 56(84) bytes of data.\n64 bytes from parasilo-17.rennes.grid5000.fr (172.16.97.17): icmp_seq=1 ttl=64 time=0.105 ms\n64 bytes from parasilo-17.rennes.grid

## Cleaning

In [49]:
provider.destroy()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1815745 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Killing the job (rennes, 1815745)
